# 08 - Computed Fields & Server Functions

## Use Case: Order Totals, Display Names & Server Timestamps

SurrealDB can compute field values server-side using `DEFINE FIELD ... VALUE <expression>`.
This is useful for:

- **Derived fields** — Full name from first + last, total from price * quantity
- **Server timestamps** — `time::now()` for created_at / updated_at without client clock issues
- **Hashing / encryption** — `crypto::argon2::generate()` for password hashing server-side

The ORM also provides `SurrealFunc` for embedding raw SurrealQL expressions in
save/update operations, and `call_function()` for invoking custom stored functions.

The SDK provides a **Typed Functions API** (`db.fn.*`) for calling SurrealDB built-in
functions directly with IDE autocompletion and type hints.

This notebook covers (from simple to complex):

1. **Typed Functions API** — `db.fn.math.*`, `db.fn.string.*`, `db.fn.crypto.*`, etc.
2. **Computed Fields** — Declarative server-computed values
3. **SurrealFunc** — Inline server-side expressions
4. **extra_vars** — Bound parameters for SurrealFunc
5. **call_function()** — Invoke custom stored functions
6. **Typed Function Calls** — `call_function()` with `return_type` for Pydantic/dataclass results
7. **raw_query()** — Execute arbitrary SurrealQL

## Prerequisites

- SurrealDB running locally (`docker run --rm -p 8000:8000 surrealdb/surrealdb:latest start --user root --pass root`)
- Python packages installed (`uv sync` in the project root)

In [ ]:
# Setup: add project root to path and configure the connection
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
from dotenv import load_dotenv
load_dotenv()

from src.surreal_orm import SurrealDBConnectionManager

SurrealDBConnectionManager.set_connection(
    os.getenv("SURREALDB_URL", "ws://localhost:8000"),
    os.getenv("SURREALDB_USER", "root"),
    os.getenv("SURREALDB_PASS", "root"),
    os.getenv("SURREALDB_NAMESPACE", "ns"),
    os.getenv("SURREALDB_DATABASE", "db"),
)

## 1. Typed Functions API

The SDK exposes SurrealDB built-in functions via a fluent `db.fn.*` interface.
Every call is **awaitable** and translates to `RETURN namespace::function(args);`.

Typed namespaces provide **IDE autocompletion** and **docstrings** for:

- `db.fn.math` — `sqrt`, `pow`, `abs`, `ceil`, `floor`, `round`, `sum`, `mean`, ...
- `db.fn.string` — `len`, `uppercase`, `lowercase`, `concat`, `split`, `trim`, ...
- `db.fn.time` — `now`, `day`, `month`, `year`, `hour`, `format`, ...
- `db.fn.array` — `len`, `append`, `distinct`, `flatten`, `sort`, `reverse`, ...
- `db.fn.crypto` — `sha256`, `sha512`, `md5`, `argon2.generate`, `bcrypt.compare`, ...

Any function not explicitly typed still works via **dynamic attribute resolution**
(e.g., `db.fn.geo.distance(...)` maps to `geo::distance()`).

Custom user-defined functions are automatically prefixed with `fn::`:
`db.fn.my_function(...)` maps to `fn::my_function()`.

In [ ]:
# Get the SDK client — this is how you access db.fn.*
from src.surreal_orm import SurrealDBConnectionManager

db = await SurrealDBConnectionManager.get_client()
print(f"Connected: {db}")
print(f"Function namespace: {db.fn}")

In [ ]:
# Math functions — sqrt, pow, abs, ceil, floor, round, sum, mean, ...
sqrt_result = await db.fn.math.sqrt(144)
pow_result = await db.fn.math.pow(2, 10)
abs_result = await db.fn.math.abs(-42)
ceil_result = await db.fn.math.ceil(3.14)

print(f"math::sqrt(144)   = {sqrt_result}")
print(f"math::pow(2, 10)  = {pow_result}")
print(f"math::abs(-42)    = {abs_result}")
print(f"math::ceil(3.14)  = {ceil_result}")

In [ ]:
# String functions — uppercase, lowercase, len, concat, split, ...
upper = await db.fn.string.uppercase("hello world")
length = await db.fn.string.len("SurrealDB")
joined = await db.fn.string.concat("Hello", ", ", "World", "!")

print(f"string::uppercase('hello world') = '{upper}'")
print(f"string::len('SurrealDB')         = {length}")
print(f"string::concat(...)              = '{joined}'")

In [ ]:
# Time functions — now, day, month, year, ...
now = await db.fn.time.now()
print(f"time::now() = {now}")
print(f"Type: {type(now).__name__}")
print("The timestamp comes from SurrealDB's server clock.")

In [ ]:
# Crypto functions — sha256, sha512, md5, argon2, bcrypt, ...
sha = await db.fn.crypto.sha256("Hello SurrealDB")
md5 = await db.fn.crypto.md5("Hello SurrealDB")

print(f"crypto::sha256('Hello SurrealDB') = {sha}")
print(f"crypto::md5('Hello SurrealDB')    = {md5}")

# Argon2 password hashing (sub-namespace)
hashed = await db.fn.crypto.argon2.generate("my_secret_password")
print(f"\ncrypto::argon2::generate(...)     = {hashed[:50]}...")

In [ ]:
# Array functions — len, distinct, flatten, sort, reverse, ...
arr_len = await db.fn.array.len([1, 2, 3, 4, 5])
distinct = await db.fn.array.distinct([1, 2, 2, 3, 3, 3])
flattened = await db.fn.array.flatten([[1, 2], [3, 4], [5]])

print(f"array::len([1,2,3,4,5])              = {arr_len}")
print(f"array::distinct([1,2,2,3,3,3])       = {distinct}")
print(f"array::flatten([[1,2],[3,4],[5]])     = {flattened}")

In [ ]:
# Dynamic namespace — any SurrealDB function works, even without typed hints
# Built-in namespaces resolve to namespace::function
rand_int = await db.fn.rand.int(1, 100)
print(f"rand::int(1, 100) = {rand_int}")

# You can also inspect the generated SQL before executing
from src.surreal_sdk.functions import FunctionCall
call = db.fn.math.sqrt(256)
sql, params = call.to_sql()
print(f"\nGenerated SQL: {sql}")
print(f"Parameters:    {params}")

## 2. Computed Fields

Computed fields are defined in the model using `Computed[T] = Computed("expression")`.
They are:

- **Read-only in Python** — They default to `None` and are populated by SurrealDB after write
- **Auto-excluded from writes** — The ORM knows not to send them in INSERT/UPDATE
- **Computed server-side** — SurrealDB evaluates the expression on every write

This requires a `DEFINE FIELD ... VALUE <expression>` in SurrealDB, which
the migration system generates automatically.

In [ ]:
# Define a model with computed fields
from src.surreal_orm import BaseSurrealModel, SurrealConfigDict
from src.surreal_orm.fields import Computed


class UserProfile(BaseSurrealModel):
    model_config = SurrealConfigDict(table_name="user_profiles")

    id: str | None = None
    first_name: str
    last_name: str
    # Computed: SurrealDB concatenates first + last name on every save
    full_name: Computed[str] = Computed("string::concat(first_name, ' ', last_name)")


print("UserProfile model defined with computed 'full_name' field")

In [ ]:
# Set up the computed field definition in SurrealDB
# In production, this would be handled by the migration system
await UserProfile.raw_query(
    "DEFINE FIELD full_name ON user_profiles VALUE string::concat(first_name, ' ', last_name)"
)

print("DEFINE FIELD full_name created with VALUE expression.")

In [ ]:
# Create a user profile — full_name is computed by SurrealDB
profile = UserProfile(first_name="Alice", last_name="Johnson")
print(f"Before save: full_name = {profile.full_name}")

await profile.save()

# Fetch back from DB to see the computed value
profile = await UserProfile.objects().get(profile.id)
print(f"After save:  full_name = '{profile.full_name}'")
print("The full_name was computed server-side by SurrealDB!")

In [ ]:
# Update the last name — full_name recomputes automatically
await profile.merge(last_name="Williams")

profile = await UserProfile.objects().get(profile.id)
print(f"After name change: full_name = '{profile.full_name}'")

## 3. SurrealFunc — Server-Side Expressions

`SurrealFunc("expression")` lets you embed raw SurrealQL expressions in `save()`
and `merge()` calls. The expression is passed directly to SurrealDB instead of
being treated as a Python value.

Use cases:
- **`time::now()`** — Server timestamps (avoids client/server clock skew)
- **`math::sum()`** — Server-side calculations
- **`crypto::argon2::generate()`** — Password hashing without exposing the hash to Python

For `save()`, use the `server_values` parameter.
For `merge()`, SurrealFunc values are detected automatically.

In [ ]:
# Define a model that uses server timestamps
from src.surreal_orm import SurrealFunc
from datetime import datetime


class Player(BaseSurrealModel):
    model_config = SurrealConfigDict(table_name="players")

    id: str | None = None
    name: str
    seat_position: int = 0
    joined_at: datetime | None = None
    last_ping: datetime | None = None


print("Player model defined")

In [ ]:
# Use SurrealFunc in save() to set a server-side timestamp
# The server_values parameter accepts SurrealFunc expressions
player = Player(name="Alice", seat_position=1)
await player.save(server_values={
    "joined_at": SurrealFunc("time::now()"),
})

# Fetch back to see the server-generated timestamp
player = await Player.objects().get(player.id)
print(f"Player: {player.name}")
print(f"Joined at (set by server): {player.joined_at}")
print("The timestamp was set by SurrealDB's clock, not the Python client.")

In [ ]:
# Use SurrealFunc in merge() — SurrealFunc values are auto-detected
await player.merge(last_ping=SurrealFunc("time::now()"))

player = await Player.objects().get(player.id)
print(f"Last ping (server timestamp): {player.last_ping}")

## 4. extra_vars — Bound Parameters for SurrealFunc

When a SurrealFunc expression references a parameter (e.g., `$password`), you
need to provide the value separately. The `extra_vars` parameter binds these
values safely as query variables.

This is especially useful for:
- **Password hashing** — Pass the raw password as a variable, hash it server-side
- **Complex expressions** — Any SurrealFunc that references `$variable`

In [ ]:
# Define a model for demonstrating extra_vars with password hashing
class Account(BaseSurrealModel):
    model_config = SurrealConfigDict(table_name="accounts")

    id: str | None = None
    username: str
    password_hash: str = ""


# Use SurrealFunc + extra_vars to hash a password server-side
raw_password = "my_secret_password_123"

account = Account(username="alice")
await account.save(
    # SurrealFunc references $password — a query variable, not a Python value
    server_values={"password_hash": SurrealFunc("crypto::argon2::generate($password)")},
    # extra_vars binds the actual value to the $password variable
    extra_vars={"password": raw_password},
)

# Fetch back to see the hashed password
account = await Account.objects().get(account.id)
print(f"Username: {account.username}")
print(f"Password hash: {account.password_hash[:60]}...")
print("The raw password never appears in the query — only the $password variable.")

## 5. call_function() — Custom Stored Functions

SurrealDB supports user-defined functions with `DEFINE FUNCTION`. You can call
these from the ORM using `call_function()`.

This is available on both:
- **`SurrealDBConnectionManager.call_function()`** — Global access
- **`Model.call_function()`** — On any model class (uses the same connection)

Use cases:
- Complex business logic that should run server-side
- Atomic operations that span multiple tables
- Functions that benefit from SurrealDB's graph traversal

In [ ]:
# Define a custom function in SurrealDB
await Player.raw_query("""
DEFINE FUNCTION fn::greet($name: string) {
    RETURN string::concat('Hello, ', $name, '! Welcome to the game.');
};
""")

print("Custom function fn::greet defined in SurrealDB.")

In [ ]:
# Call the custom function from the ORM
result = await Player.call_function("greet", params={"name": "Alice"})
print(f"Function result: {result}")

In [ ]:
# Define a more practical function: count active players at a table
await Player.raw_query("""
DEFINE FUNCTION fn::count_players($table_id: string) {
    LET $count = (SELECT count() FROM players WHERE seat_position > 0 GROUP ALL);
    RETURN $count[0].count OR 0;
};
""")

# Create a few more players
await Player(name="Bob", seat_position=2).save()
await Player(name="Charlie", seat_position=3).save()

# Call the function
count = await Player.call_function("count_players", params={"table_id": "game:1"})
print(f"Active players: {count}")

## 6. Typed Function Calls — `return_type` for Structured Results

When a custom function returns a structured object (dict/record), you can pass
a **Pydantic model** or **dataclass** as `return_type` to get a fully typed result
instead of a raw dict.

This is especially useful for:
- **Business logic functions** — e.g., voting, scoring, inventory checks
- **Aggregation functions** — returning computed stats as typed objects
- **API boundaries** — ensuring function results match expected schemas

Both `Model.call_function()` and `SurrealDBConnectionManager.call_function()`
accept the `return_type` parameter.

In [ ]:
# Define a Pydantic model for the function's return value
from pydantic import BaseModel


class PlayerStats(BaseModel):
    total: int
    ready: int
    avg_seat: float


# Define a custom function that returns a structured object
await Player.raw_query("""
DEFINE FUNCTION fn::player_stats() {
    LET $players = SELECT * FROM players;
    LET $total = array::len($players);
    LET $ready = array::len(
        SELECT * FROM players WHERE seat_position > 0
    );
    LET $avg = math::mean($players.seat_position) OR 0;
    RETURN { total: $total, ready: $ready, avg_seat: $avg };
};
""")

print("Custom function fn::player_stats defined.")

In [ ]:
# Call WITHOUT return_type — returns a raw dict
raw_result = await Player.call_function("player_stats")
print(f"Without return_type: {raw_result}")
print(f"  Type: {type(raw_result).__name__}")

# Call WITH return_type — returns a typed Pydantic model
stats = await Player.call_function("player_stats", return_type=PlayerStats)
print(f"\nWith return_type:    {stats}")
print(f"  Type:  {type(stats).__name__}")
print(f"  total: {stats.total}")
print(f"  ready: {stats.ready}")
print(f"  avg_seat: {stats.avg_seat}")
print("\nWith return_type you get IDE autocompletion, type checking, and Pydantic validation.")

## 7. raw_query() — Execute Arbitrary SurrealQL

`raw_query()` is the escape hatch for anything the ORM does not cover.
It executes arbitrary SurrealQL and returns raw results.

Use cases:
- Complex graph traversals that the QuerySet API cannot express
- Administrative queries (DEFINE TABLE, REMOVE INDEX, etc.)
- One-off data migrations
- Debugging and exploration

**Always use `variables` for user input** to prevent SurrealQL injection.

In [ ]:
# Simple raw query
results = await Player.raw_query("SELECT * FROM players ORDER BY name")
print("All players (raw query):")
for r in results:
    print(f"  {r}")

In [ ]:
# Parameterized raw query — safe from injection
results = await Player.raw_query(
    "SELECT * FROM players WHERE seat_position >= $min_seat",
    variables={"min_seat": 2},
)
print(f"Players at seat 2+: {len(results)} found")
for r in results:
    print(f"  {r.get('name', 'unknown')} at seat {r.get('seat_position', '?')}")

In [ ]:
# Use raw_query for database administration
info = await Player.raw_query("INFO FOR DB")
print("Database info:")
print(f"  {info}")

## Cleanup

Remove all data and function definitions created during this notebook.

In [ ]:
# Clean up all tables and function definitions
await UserProfile.raw_query("DELETE FROM user_profiles")
await Player.raw_query("DELETE FROM players")
await Account.raw_query("DELETE FROM accounts")

# Remove custom function definitions
await Player.raw_query("REMOVE FUNCTION fn::greet")
await Player.raw_query("REMOVE FUNCTION fn::count_players")
await Player.raw_query("REMOVE FUNCTION fn::player_stats")

# Remove computed field definition
await UserProfile.raw_query("REMOVE FIELD full_name ON user_profiles")

print("Cleanup complete: all data, functions, and field definitions removed.")

## Summary

| Feature | Syntax | Use Case |
|---------|--------|----------|
| **Typed Functions API** | `db = await ...get_client(); await db.fn.math.sqrt(16)` | Call SurrealDB built-in functions with IDE autocompletion |
| **Computed Fields** | `Computed[T] = Computed("expr")` | Derived values computed by SurrealDB on every write |
| **SurrealFunc (save)** | `save(server_values={"f": SurrealFunc("...")})` | Server-side expressions during create/update |
| **SurrealFunc (merge)** | `merge(field=SurrealFunc("..."))` | Server-side expressions during partial update |
| **extra_vars** | `save(..., extra_vars={"k": v})` | Bind parameters referenced by SurrealFunc |
| **call_function()** | `Model.call_function("name", params={})` | Invoke custom stored functions |
| **Typed return** | `Model.call_function("name", return_type=MyModel)` | Get Pydantic/dataclass results instead of raw dicts |
| **raw_query()** | `Model.raw_query("SQL", variables={})` | Execute arbitrary SurrealQL |

**Key principle**: Keep complex logic on the server. Use Computed fields for
declarative expressions, SurrealFunc for imperative operations, the Typed Functions
API for direct built-in function calls, and call_function() with `return_type` for
reusable stored procedures with structured results.